<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/OPENAI_FT_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai -q
!pip install python-dotenv -q
!pip install tiktoken -q
!pip install colab-env -q
!pip install datasets -q

In [ ]:
import colab_env
import os
openai_api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
from openai import OpenAI
client = OpenAI()

modellist=client.models.list()
modellist.data

[Model(id='o1-pro', created=1742251791, object='model', owned_by='system'),
 Model(id='gpt-4o-audio-preview-2024-12-17', created=1734034239, object='model', owned_by='system'),
 Model(id='o1-pro-2025-03-19', created=1742251504, object='model', owned_by='system'),
 Model(id='gpt-4o-realtime-preview-2024-12-17', created=1733945430, object='model', owned_by='system'),
 Model(id='dall-e-3', created=1698785189, object='model', owned_by='system'),
 Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'),
 Model(id='gpt-4o-audio-preview-2024-10-01', created=1727389042, object='model', owned_by='system'),
 Model(id='gpt-4-0314', created=1687882410, object='model', owned_by='openai'),
 Model(id='gpt-4o-realtime-preview-2024-10-01', created=1727131766, object='model', owned_by='system'),
 Model(id='gpt-4o-transcribe', created=1742068463, object='model', owned_by='system'),
 Model(id='gpt-4o-mini-transcribe', created=1742068596, object='model', owned_by='system'),
 Model(id='g

In [ ]:
from datasets import load_dataset

dataset = load_dataset("frankmorales2020/flight_plan_waypoints")

In [ ]:
dataset['train'][0]

{'input': 'Calculate the waypoints from SIN to CUN. Departure: 2024-06-19, Aircraft: Airbus A320, Weather: Partly Cloudy',
 'label': 7,
 'distance': 4190.223965150766,
 'distance_category': 'long',
 'waypoints': [[25.0000001, -107.5000001],
  [13.075850948259008, -87.47663616549106],
  [19.819922201759134, -98.80146877121005],
  [9.260386709076178, -81.06960216515675],
  [10.868386703521901, -83.76980061837503],
  [11.748790002704872, -85.24819840125659],
  [13.307169797892762, -87.86507323149776],
  [14.392416690114095, -89.68745009086217],
  [5.1096596, -74.0995854]],
 'waypoint_names': ['SIN',
  'Choluteca',
  'Santo Domingo Aztacameca',
  'Veraguas',
  'San Juan del Norte',
  'Acoyapa',
  'Pasaje La Cruz',
  'El Pito',
  'CUN']}

In [ ]:
import openai
import json
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("frankmorales2020/flight_plan_waypoints")

# Convert to JSONL format with 'input', 'label', 'waypoints', and 'waypoint_names'
training_data = []
for row in dataset["train"]:
    data_point = {
        "prompt": row["input"],
        "completion": row["label"],
        "waypoints": row["waypoints"],
        "waypoint_names": row["waypoint_names"],
    }
    training_data.append(data_point)

# Save the training data to a JSONL file
with open("training_data.jsonl", "w") as f:
    for data_point in training_data:
        f.write(json.dumps(data_point) + "\n")

print("Dataset converted to JSONL and saved to training_data.jsonl")

Dataset converted to JSONL and saved to training_data.jsonl


In [ ]:
{"prompt": "Calculate the waypoints from SIN to CUN. Departure: 2024-06-19, Aircraft: Airbus A320, Weather: Partly Cloudy", "completion": 7, "waypoints": [[25.0000001, -107.5000001], [13.075850948259008, -87.47663616549106], [19.819922201759134, -98.80146877121005], [9.260386709076178, -81.06960216515675], [10.868386703521901, -83.76980061837503], [11.748790002704872, -85.24819840125659], [13.307169797892762, -87.86507323149776], [14.392416690114095, -89.68745009086217], [5.1096596, -74.0995854]], "waypoint_names": ["SIN", "Choluteca", "Santo Domingo Aztacameca", "Veraguas", "San Juan del Norte", "Acoyapa", "Pasaje La Cruz", "El Pito", "CUN"]}

## data split

In [ ]:
from sklearn.model_selection import train_test_split

# 1. Load and prepare the training data
# ----------------------------------------
with open("/content/training_data.jsonl", "r") as f:
    training_data = [json.loads(line) for line in f]

# 2. Split data into training and evaluation sets
# ----------------------------------------
train_data, eval_data = train_test_split(training_data, test_size=0.2, random_state=42)


In [ ]:
len(train_data)
len(eval_data)

400

## fine tuning

In [ ]:
!pip install openai -q
!pip install python-dotenv -q
!pip install tiktoken -q
!pip install colab-env -q
!pip install datasets -q
!pip install scikit-learn -q

import json
import io
import os
import time
from datasets import load_dataset
from openai import OpenAI
import openai
from sklearn.model_selection import train_test_split

# Set your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()

# 1. Load and prepare the training data with "messages" key
# ---------------------------------------------------------
with open("/content/training_data.jsonl", "r") as f:
    training_data = [json.loads(line) for line in f]

# Restructure data to include "messages"
formatted_training_data = []
for data_point in training_data:
    formatted_data_point = {
        "messages": [
            {"role": "system", "content": "You are an AI assistant that helps calculate flight plan waypoints."},
            {"role": "user", "content": data_point["prompt"]},
            {"role": "assistant", "content": str(data_point["completion"])}
        ]
    }
    formatted_training_data.append(formatted_data_point)

# 2. Split data into training and evaluation sets
# ----------------------------------------
train_data, eval_data = train_test_split(formatted_training_data, test_size=0.2, random_state=42)


# 3. Create validation dataset
#-------------------------------------------
# Assuming 'eval_data' is your validation data
validation_data_file = io.BytesIO()
for data_point in eval_data:
    validation_data_file.write((json.dumps(data_point) + "\n").encode('utf-8'))
validation_data_file.seek(0)

validation_file_id = client.files.create(
    file=validation_data_file, purpose="fine-tune"
).id


# 4. Fine-tuning with Validation Set
# ----------------------------------------
# Create an in-memory file-like object for the training data
training_data_file = io.BytesIO()
for data_point in train_data:
    training_data_file.write((json.dumps(data_point) + "\n").encode('utf-8'))
training_data_file.seek(0)  # Reset file pointer to the beginning

# Upload the in-memory file to OpenAI
file_id = client.files.create(
    file=training_data_file, purpose="fine-tune"
).id

# Define fine-tuning parameters, including validation_file
fine_tuning_params = {
    "training_file": file_id,
    "model": "gpt-3.5-turbo",
    "validation_file": validation_file_id,  # Include validation file
}

# Create the fine-tuning job
fine_tuning_job = client.fine_tuning.jobs.create(**fine_tuning_params)
fine_tuning_job_id = fine_tuning_job.id

print(f"Fine-tuning job created with ID: {fine_tuning_job.id}")

# 5. Monitor fine-tuning job
# -----------------------------
while True:
    fine_tuning_job = client.fine_tuning.jobs.retrieve(fine_tuning_job_id)
    status = fine_tuning_job.status
    print(f"Fine-tuning job status: {status}")

    if status == "succeeded":
        print("Fine-tuning completed successfully!")
        fine_tuned_model_id = fine_tuning_job.fine_tuned_model
        print(f"Fine-tuned model ID: {fine_tuned_model_id}")
        break
    elif status in ["failed", "cancelled"]:
        print(f"Fine-tuning job {status}.")
        break
    else:  # pending, running, etc.
        time.sleep(60)  # Check every 60 seconds

Fine-tuning job created with ID: ftjob-g9Ps0nEmhDUB1g8PBZfojptB
Fine-tuning job status: validating_files
Fine-tuning job status: validating_files
Fine-tuning job status: running
Fine-tuning job status: running
Fine-tuning job status: running
Fine-tuning job status: running
Fine-tuning job status: running
Fine-tuning job status: running
Fine-tuning job status: running
Fine-tuning job status: running
Fine-tuning job status: running
Fine-tuning job status: running
Fine-tuning job status: running
Fine-tuning job status: running
Fine-tuning job status: running
Fine-tuning job status: running
Fine-tuning job status: running
Fine-tuning job status: running
Fine-tuning job status: running
Fine-tuning job status: running
Fine-tuning job status: running
Fine-tuning job status: running
Fine-tuning job status: running
Fine-tuning job status: running
Fine-tuning job status: running
Fine-tuning job status: running
Fine-tuning job status: running
Fine-tuning job status: running
Fine-tuning job status

## evaluation

In [ ]:
!pip install tqdm -q
from tqdm import tqdm

In [ ]:
# Evaluation
# ----------------------------------------
# Replace with your fine-tuned model ID
# ftjob-ePiCspTiSe6K9sm7c0UE05qp
fine_tuned_model_id = "ft:gpt-3.5-turbo-0125:xamrysoft::BFqZ9l56"

predictions = []
#for data_point in tqdm(eval_data, desc="Evaluating", unit="data point"):  # Add tqdm here
for data_point in tqdm(eval_data, desc="Evaluating", unit="data point", miniters=1):  # Add miniters=1 here


    response = client.chat.completions.create(
        model=fine_tuned_model_id,
        messages=data_point["messages"]
    )

    prediction = response.choices[0].message.content
    print('\n')
    print(f"Prediction: {prediction}")
    predictions.append(prediction)

correct_predictions = 0
total_predictions = 0

for data_point, prediction in zip(eval_data, predictions):
    # Assuming the last message in 'messages' is the ground truth
    expected_output = str(data_point["messages"][-1]["content"])
    print(f"Expected: {expected_output}")

    if prediction == expected_output:
        print(f"Prediction: {prediction}, Expected: {expected_output}")
        correct_predictions += 1
    total_predictions += 1

accuracy = correct_predictions / total_predictions
print(f"Accuracy: {accuracy}")